In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from time import strptime

pd.set_option('display.max_rows', None)  # 모든 행 출
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.expand_frame_repr', None)  # 긴 데이터 프레임 줄바꿈 없이 출력

symbols = ['SPY', 'IWM']
start_date = '2015-12-01'
end_date = '2016-12-31'

In [2]:
# 배당 데이터 가져오기
def get_dividends(symbols:list):
    result_dfs = {}

    for symbol in symbols:
        info = yf.Ticker(symbol)
        df = info.actions.reset_index()
        df['Date'] = df['Date'].dt.date # 년월일만 남김
        df['Symbol'] = symbol
        df = df[['Date','Symbol','Dividends']]
        result_dfs[symbol]=df

    return result_dfs

In [3]:
# 주가 데이터 가져오기
def get_stock_data(symbols:list, start_date, end_date):
    
    result_dfs = {}
    
    # 로딩하는건 progress = False 하면 해제됨
    df = yf.download(tickers=symbols, start=start_date, end=end_date, progress=False, auto_adjust=False)

    # 배당 데이터 가져오기
    dividends_dfs = get_dividends(symbols= symbols)

    for symbol in symbols:
        # 특정 심볼 선택
        df_symbol = df.loc[:, df.columns.get_level_values(1) == symbol]

        # 멀티 column의 심볼들 제거 후, 심볼 추가
        df_symbol.columns = df_symbol.columns.droplevel(1)
        df_symbol = df_symbol.reset_index()
        df_symbol .columns.name = None

        # 심볼 추가 및 column 위치 조정
        df_symbol['Symbol'] = symbol
        columns = [col for col in df_symbol.columns if col != 'Date' and col != 'Symbol']
        columns = ['Date', 'Symbol'] + columns 
        df_symbol = df_symbol[columns]

        # Date값을 datetime 으로 변경
        df_symbol['Date'] = pd.to_datetime(df_symbol['Date'])

        df_dividends = dividends_dfs[symbol]
        df_dividends['Date'] = pd.to_datetime(df_dividends['Date'])

        # 배당데이터 병합
        df_symbol = df_symbol.merge(df_dividends, on=['Date','Symbol'], how = 'left')
        df_symbol.fillna(0, inplace=True)
        result_dfs[symbol] = df_symbol

    return result_dfs

In [4]:
# 종가 데이터 가져오기
def get_stock_data_close(symbols:list, start_date, end_date=None, include_diviend=False, key = 'Close'):
    
    result_dfs = {}

    symbol_dfs = get_stock_data(symbols=symbols, start_date=start_date,end_date=end_date)
    
    for symbol, df_symbol in symbol_dfs.items():

        if key == 'Adj Close':
           result_close = df_symbol['Adj Close']
           condition = 'Adj Close'
        
        elif key == 'Close':
            if include_diviend == True:
                result_close = df_symbol['Close'] + df_symbol['Dividends']
                condition = 'Close + Dividends'
            else:
                result_close = df_symbol['Close']
                condition = 'Close'

        df_symbol['Result Close'] = result_close
        df_symbol['Condition'] = condition
        df_symbol = df_symbol[['Date','Symbol','Adj Close', 'Close', 'Dividends','Result Close','Condition']]

        result_dfs[symbol] = df_symbol

    return result_dfs


# get_stock_data_close(symbols=symbols, start_date=start_date, include_diviend=True)

In [26]:
def get_stock_data_with_ma(symbols:list, start_date, end_date, mas, type='ma_day'):

    result_dfs = {}

    if len(mas) <= 0:
        print('mas count is 0!')
        return
    
    if start_date != None:
       start_date = datetime.strptime(start_date, "%Y-%m-%d") if isinstance(start_date, str) else start_date
    
    if end_date != None:
       end_date = datetime.strptime(end_date, "%Y-%m-%d") if isinstance(end_date, str) else end_date

    sort_mas = sorted(mas)

    # 일별이동평균
    if type == 'ma_day':
        prev_day = sort_mas[-1]
        prev_date = start_date - relativedelta(days = prev_day*1.5)

    
    # 월별이동평균 - 해당월의 마지막날 데이터의 종가 기준으로 평균 값 구함.
    elif type == 'ma_month':
        prev_day = sort_mas[-1]*25
        prev_date = start_date -relativedelta(days=prev_day*1.5)

    # 월말 데이터 가져오기
    symbol_dfs = get_stock_data_close(symbols=symbols, start_date=prev_date, end_date=end_date, include_diviend=True, key= 'Close')

    for symbol, df_symbol in symbol_dfs.items():
        df = df_symbol.copy()

        if type == 'ma_day':
            for ma in mas:
                df[f'SMA_{ma}'] = df['Result Close'].rolling(window=ma).mean()

        elif type == 'ma_month':
            monthly_df = df.groupby(df['Date'].dt.to_period('M')).apply(lambda x: x.iloc[-1]).reset_index(drop=True)


            new_columes = []
            for ma in mas:
                col = f'MMA_{ma}'
                monthly_df[col] = monthly_df['Result Close'].rolling(window=ma).mean()
                new_columes.append(col)
                
            
            df = df.merge(monthly_df[['Date', 'Symbol'] + new_columes], on=['Date', 'Symbol'], how='left')
            
            
        # 기간범위 체크
        df = df[df['Date'] >= start_date]
        if end_date is not None:
            df = df[df['Date'] <= end_date]

        result_dfs[symbol] = df

    return result_dfs
            
get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=None,mas=[10], type='ma_day')

{'SPY':            Date Symbol   Adj Close       Close  Dividends  Result Close          Condition      SMA_10
 10   2015-12-01    SPY  179.048813  210.679993      0.000    210.679993  Close + Dividends  208.873001
 11   2015-12-02    SPY  177.221573  208.529999      0.000    208.529999  Close + Dividends  209.179001
 12   2015-12-03    SPY  174.740005  205.610001      0.000    205.610001  Close + Dividends  208.867001
 13   2015-12-04    SPY  178.147964  209.619995      0.000    209.619995  Close + Dividends  208.974001
 14   2015-12-07    SPY  177.068665  208.350006      0.000    208.350006  Close + Dividends  208.878001
 15   2015-12-08    SPY  175.878876  206.949997      0.000    206.949997  Close + Dividends  208.666000
 16   2015-12-09    SPY  174.510559  205.339996      0.000    205.339996  Close + Dividends  208.264999
 17   2015-12-10    SPY  174.961014  205.869995      0.000    205.869995  Close + Dividends  207.919998
 18   2015-12-11    SPY  171.570038  201.880005      0.00

In [ ]:
# 월말 데이터 가져오기
def filter_close_last_month(symbol_dfs):
    result_dfs = {}

    for symbol, df_symbol in symbol_dfs.items():
        monthly_df = df_symbol.groupby(df_symbol['Date'].dt.to_period('M')).apply(lambda x: x.iloc[-1]).reset_index(drop=True)
        
        result_dfs[symbol] = monthly_df
    
    return result_dfs

In [7]:
# 수익률 구하기
def get_profit_ratio(symbol_dfs: dict, init_balance: float = 10000) -> dict:
    result_dfs = {}

    for symbol, df_symbol in symbol_dfs.items():
        df_symbol = df_symbol.copy()

        # 컬럼 초기화
        df_symbol['Profit Ratio'] = 0.0
        df_symbol['Balance'] = 0.0

        # 초기값 설정
        df_symbol.loc[0, 'Balance'] = init_balance
        df_symbol.loc[0, 'Profit Ratio'] = 0.0

        for i in range(1, len(df_symbol)):
            prev_close = df_symbol.loc[i - 1, 'Result Close']
            curr_close = df_symbol.loc[i, 'Result Close']

            if pd.notna(prev_close) and prev_close != 0:
                profit_ratio = (curr_close - prev_close) / prev_close
            else:
                profit_ratio = 0.0

            df_symbol.loc[i, 'Profit Ratio'] = round(profit_ratio * 100, 2)
            df_symbol.loc[i, 'Balance'] = round(df_symbol.loc[i - 1, 'Balance'] * (1 + profit_ratio), 2)

        result_dfs[symbol] = df_symbol
        display(df_symbol)

    return result_dfs

# df = get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=end_date,mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# get_profit_ratio(df)

In [8]:
def get_profit_ratio_avg(symbol_dfs:dict, intervals=[1,3,6]):

    result_dfs = {}

    for symbol, df in symbol_dfs.items():
        
        interval_dfs ={}
        for interval in intervals:
            copy_df = df.copy()
            shift_df = copy_df.shift(interval)
            
            copy_df['Interval'] = interval

            profit_value = copy_df['Result Close'] - shift_df['Result Close']

            copy_df['Profit Value'] = profit_value
            copy_df['Profit Ratio'] = ((profit_value/shift_df['Result Close'])*100).round(2)

            interval_dfs[interval] = copy_df
            
        
        avg_profit_ratios = sum(df_interval['Profit Ratio'] for df_interval in interval_dfs.values()) / len(intervals)
        
        new_df = df.copy()
        new_df['Avg Profit Ratio'] = avg_profit_ratios
        new_df['Interval'] = [intervals] * len(df)

        result_dfs[symbol] = new_df

    return result_dfs

# df = get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=end_date,mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# get_profit_ratio_avg(df)

In [9]:
def merge_to_dfs(symbol_dfs:dict):
    all_dfs = []
    for symbol, df_symbol in symbol_dfs.items():
        df = df_symbol.copy()
        all_dfs.append(df)

    merged_df = pd.concat(all_dfs, ignore_index=True)
    group_df = merged_df.groupby('Date').agg(lambda x: list(x)).reset_index()

    for col in group_df.columns:
        group_df[col] = group_df[col].apply(
            lambda x: [round(val, 2) if isinstance(val, (int, float)) else val for val in x] if isinstance(x, list) else x
        )

    return group_df

In [ ]:
# 리벨런싱(균등) - 연간 N회 리벨런싱. 균등하게
def strategy_evenly(symbol_dfs: dict, ratios:list, interval:int = 12, init_balance = 10000):

    if len(symbol_dfs) != len(ratios):
        print('심볼이랑, 가중치랑 길이가 다름!')
        return
    
    if sum(ratios) != 100:
        print(f'가중치 값이 다름! : {sum(ratios)}')
        return

    df = merge_to_dfs(symbol_dfs)
    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition']]
    df = df[columns]

    total_ratio = sum(ratios)
    start_balances = [round((r / total_ratio) * init_balance, 2) for r in ratios]
    end_balances = [0]*len(ratios)

    # 초기값 설정
    df['End Balance'] = None
    df['Restart Balance'] = None
    df['Balance'] = None
    df['Can Rebalance'] = False

    df.at[0, 'End Balance'] = end_balances
    df.at[0, 'Restart Balance'] = start_balances
    df.at[0, 'Balance'] = init_balance

    for i in range(1, len(df)):
        # interval 값만큼 건너뛰고 리벨런싱
        # 각 종목의 증감률을 Result Close에서 계산
        result_close = df.at[i, 'Result Close']
        change_factors = [result_close[j]/ df.at[i-1,'Result Close'][j] for j in range(len(result_close))]

        end_balances = [round(df.at[i-1, 'Restart Balance'][j] * change_factors[j], 2) for j in range(len(change_factors))]
        df.at[i, 'End Balance'] = end_balances

        next_balance = sum(end_balances)
        df.at[i, 'Balance'] = int(next_balance)
        
        if i % interval == 0:
            restart_balance = [round((r/100)*next_balance,2) for r in ratios]
            df.at[i, 'Restart Balance'] = restart_balance
            df.at[i, 'Can Rebalance'] = True
        else:
            df.at[i, 'Restart Balance'] = end_balances
            

    return df
        

# df = get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=end_date,mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# rebalancing_evenly(df, ratios=[50,50], interval=2)

In [11]:
def get_performance(df):
    # 심볼과 기본 데이터 설정
    symbol = df['Symbol'].iloc[0]
    start_balance = df['Balance'].dropna().iloc[0]
    end_balance = df['Balance'].dropna().iloc[-1]
    start_date = pd.to_datetime(df['Date'].iloc[0])
    end_date = pd.to_datetime(df['Date'].iloc[-1])
    num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

    # 연복리 성장률 (CAGR) 계산
    years = num_months / 12
    CAGR = ((end_balance / start_balance) ** (1 / years)) - 1

    # 월간 수익률 계산
    balance_series = df['Balance'].dropna().astype(float)
    monthly_returns = balance_series.pct_change().dropna()

    # 무위험 수익률 가정 (연 2%를 월간으로 환산)
    risk_free_rate = 0.02 / 12

    # 연간화된 표준 편차 계산
    annualized_std_dev = monthly_returns.std() * np.sqrt(12)

    # Sharpe Ratio 계산
    sharpe_ratio = (monthly_returns.mean() - risk_free_rate) / monthly_returns.std() * np.sqrt(12)

    # 최대 낙폭 (MDD) 계산
    running_max = np.maximum.accumulate(balance_series)
    drawdown = (balance_series / running_max - 1).min()

    # 결과 데이터프레임 생성
    results = pd.DataFrame({
        "Symbol": [symbol],
        "Start Date" : [start_date],
        "End Date":[end_date],
        "Start Balance":[start_balance],
        "End Balance": [end_balance],
        "Annualized Return (CAGR)": [CAGR],
        "Standard Deviation": [annualized_std_dev],
        "Sharpe Ratio": [sharpe_ratio],
        "Maximum Drawdown": [drawdown]
    })

    return results


# df = get_stock_data_with_ma(symbols=['VTI','TLT','BIL','GLD'], start_date='2015-12-01', end_date='2024-12-31', mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# df = rebalancing_evenly(df, ratios=[25,25,25,25], interval=12)
# get_performance(df)
    

In [12]:
# 해리브라운의 영구포트폴리오
def harry_browne_permanent_portfolio():
    df = get_stock_data_with_ma(symbols=['VTI','TLT','BIL','GLD'], start_date='2010-12-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = strategy_evenly(df, ratios=[25,25,25,25], interval=12)
    df = get_performance(df)
    display(df)

# 레이달리오 올시즌 포트폴리오
def ray_dalio_all_seasons():
    df = get_stock_data_with_ma(symbols=['VTI','TLT','IEF','GSG','GLD'], start_date='2010-12-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = strategy_evenly(df, ratios=[30,40,15,7.5, 7.5], interval=12)
    df = get_performance(df)
    display(df)


harry_browne_permanent_portfolio()
ray_dalio_all_seasons()


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[VTI, TLT, BIL, GLD]",2010-12-31,2025-03-28,10000,18494,0.044093,0.070414,0.364702,-0.164731


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[VTI, TLT, IEF, GSG, GLD]",2010-12-31,2025-03-28,10000,17208,0.038825,0.080256,0.266162,-0.234537


In [ ]:
# 선택한 자산군 중 기간월 평균 수익률이 높은 N개를 추출
# N개를 균등하게 배분
# N개의 각 자산들의 10개월 이동평균보다 낮다면 현금으로 보유
# 월 1회 리벨런싱
def strategy_taa(symbol_dfs: dict, rank = 3, init_balance = 10000):
    
    # 'Restart Asset' 생성 함수 (조건 추가)
    def get_top_assets(row):
        profit_ratios = row['Avg Profit Ratio']
        symbols = row['Symbol']
        result_close = row['Result Close']
        mma_10 = row['MMA_10']
        
        # 높은 순으로 인덱스 정렬 후 상위 rank개 추출
        top_indices = sorted(range(len(profit_ratios)), key=lambda i: profit_ratios[i], reverse=True)[:rank]
        
        # 조건에 맞는 symbol만 추가
        selected_assets = [
            symbols[i] for i in top_indices
            if result_close[i] >= mma_10[i]
        ]
        
        return selected_assets


    for symbol, df in symbol_dfs.items():
        symbol_dfs[symbol] = df.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(symbol_dfs)
    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    df['End Balance'] = None
    df['End Cash'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_top_assets, axis=1) 
    df['Restart Balance'] = None
    
    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']
        
        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            end_cash = df.at[idx-1, 'Cash']
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance]) + end_cash

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'End Cash'] = end_cash
            df.at[idx, 'Balance'] = balance
            

        split_balance = int(balance / rank)
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance
        df.at[idx, 'Cash'] = balance
                 
    return df


# symbols = ['SPY','IWD','IWM', 'IWN','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','BWX','GLD']
# symbols = ['SPY','IWM','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','GLD']
# df = get_stock_data_with_ma(symbols=symbols, start_date='2014-11-01', end_date='2024-12-31', mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# df = get_profit_ratio_avg(df, [1,3,6,12])
# df = rebalancing_taa(df)
# display(df)
# display(get_performance(df))


In [14]:
# 공격형 TAA전략
def gtaa():
    # symbols = ['SPY','IWD','IWM', 'IWN','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','BWX','GLD']
    symbols = ['SPY','IWM','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','GLD']
    start_date = '2014-12-01'

    df = get_stock_data_with_ma(symbols=symbols, start_date=start_date, end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [1,3,6,12])
    df = strategy_taa(df, 3)
    # display(df)
    display(get_performance(df))

gtaa()

,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, IWM, MTUM, EFA, TLT, IEF, LQD, DBC, VNQ,...",2015-12-31,2025-03-28,10000,20790.0,0.082337,0.102946,0.627552,-0.136697


In [15]:
# 오리지널 듀얼모멘텀 전략
# SPY, EFA(선진국주식), AGG(미국채권) 투자
# 매월, SPY, EFA, BIL(초단기채권)의 최근 12개월 수익률 계산
# 수익률이 SPY > BIL 일 경우, SPY, EFA중 수익률 높은 ETF에 투자.
# BIL < SPY일 경우, AGG에 투자

def original_dual_momentum():

    def get_restart_asset(row):
        symbols = row['Symbol']
        profit_ratios = row['Avg Profit Ratio']
        
        # 종목 이름과 인덱스를 매핑
        symbol_to_index = {symbol: i for i, symbol in enumerate(symbols)}
        
        # 필요한 종목들이 모두 있는지 확인
        required_symbols = ['SPY', 'BIL', 'EFA', 'AGG']
        if not all(sym in symbol_to_index for sym in required_symbols):
            return []  # 조건에 맞는 종목이 없으면 빈 리스트 반환

        spy_idx = symbol_to_index['SPY']
        bil_idx = symbol_to_index['BIL']
        efa_idx = symbol_to_index['EFA']
        agg_idx = symbol_to_index['AGG']
        
        spy_ratio = profit_ratios[spy_idx]
        bil_ratio = profit_ratios[bil_idx]
        efa_ratio = profit_ratios[efa_idx]
        agg_ratio = profit_ratios[agg_idx]  # 필요하면 사용 가능

        if spy_ratio > bil_ratio:
            selected = 'SPY' if spy_ratio >= efa_ratio else 'EFA'
            return [selected]
        else:
            return ['AGG']


    init_balance = 10000
    symbols = ['SPY','EFA','AGG','BIL']
    df = get_stock_data_with_ma(symbols=symbols, start_date='2007-08-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [12])


    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_restart_asset, axis=1) 
    df['Restart Balance'] = None

    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance
        
        split_balance = int(balance / len(restart_asset))
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance

    df = get_performance(df)
    display(df)


original_dual_momentum()

,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, EFA, AGG, BIL]",2008-08-29,2025-03-28,10000,25676,0.05851,0.123904,0.360771,-0.203766


In [16]:
# 종합 듀얼 모멘텀
# 포트폴리오를 4개 파트로 나눔
# 1. 주식 - SPY, ETF(해외주식)
# 2. 채권 - LQD(회사채), HYD(미국 하이일드 채권)
# 3. 부동산 - VNQ(부동산 리츠), REM(모기지 리츠)
# 4. 불경기 - TLT, GLD
# 각 파트별 1개씩 투자함 (25% 배분)
# 최근 12개월 수익률을 비교해서 1개씩 선별함.
# 근데, 파트의 ETF 수익 모두가 BIL 수익보다 낮으면, BIL로 리벨런싱
def composite_dual_momentum():

    def get_selected_assets(row):
        symbols = row['Symbol']
        profit_ratios = row['Avg Profit Ratio']
        
        # 종목과 인덱스를 매핑
        symbol_to_index = {symbol: i for i, symbol in enumerate(symbols)}
        
        # BIL 수익률
        bil_ratio = profit_ratios[symbol_to_index['BIL']] if 'BIL' in symbol_to_index else None

        if bil_ratio is None:
            return []  # BIL 없으면 처리 안 함

        # 자산군 정의
        asset_groups = {
            'stock': ['SPY', 'EFA'],
            'bond': ['LQD', 'HYG'],
            'real_estate': ['VNQ', 'REM'],
            'recession': ['TLT', 'GLD']
        }

        selected_assets = []

        for group, candidates in asset_groups.items():
            valid_candidates = [
                (symbol, profit_ratios[symbol_to_index[symbol]])
                for symbol in candidates
                if symbol in symbol_to_index
            ]

            # 유효한 후보 없으면 해당 그룹 스킵
            if not valid_candidates:
                continue

            # 후보 중 수익률이 가장 높은 ETF 찾기
            best_symbol, best_ratio = max(valid_candidates, key=lambda x: x[1])

            # 해당 그룹의 모든 ETF 수익률이 BIL보다 낮은지 확인
            all_lower_than_bil = all(ratio < bil_ratio for _, ratio in valid_candidates)

            if all_lower_than_bil:
                selected_assets.append('BIL')
            else:
                selected_assets.append(best_symbol)

        return selected_assets


    init_balance = 10000
    
    symbols = ['SPY','EFA','LQD','HYG', 'VNQ', 'REM','TLT','GLD', 'BIL']
    df = get_stock_data_with_ma(symbols=symbols, start_date='2008-08-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [12])

    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_selected_assets, axis=1) 
    df['Restart Balance'] = None

    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance
        
        split_balance = int(balance / len(restart_asset))
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance

    # display(df)
    df = get_performance(df)
    display(df)

composite_dual_momentum()

,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, EFA, LQD, HYG, VNQ, REM, TLT, GLD, BIL]",2009-08-31,2025-03-28,10000,14431,0.023817,0.075006,0.085082,-0.140494


In [17]:
# PAA 전략
# 안전자산 비중 설정
# 12 ETF중 현재 가격이 12개월 단순이동평균보다 낮은 자산 수를 측정
# 하락추세 ETF수에 따른 안전자산 비준은 다음과 같이 설정함
# 0~6개까지 있으며, 6개이상일 경우에는 100퍼센트 안전자산 투자
# 안전자산은 미국중기국체 IEF
# 안전자산에 투잣하지 않은 금액은 상대 모멘텀으로 6개 ETF에 분산투자 함.
# 매월 말 각 ETF의 12개월 단순 이동평균을 계싼. (현재가격/12개월 이동평균) -1 이 가장 높은 6개의자산에 투자.
def paa():

    def get_selected_assets(row):
        symbols = row['Symbol']
        prices = row['Result Close']
        mas = row['MMA_10']

        selected = []

        for symbol, price, ma in zip(symbols, prices, mas):
            if symbol == 'IEF':
                continue  # IEF는 제외
            if price > ma:
                score = (price / ma) - 1
                selected.append((symbol, score))

        # 스코어 기준으로 내림차순 정렬
        selected_sorted = sorted(selected, key=lambda x: x[1], reverse=True)

        # 심볼만 리스트로 반환
        return [symbol for symbol, _ in selected_sorted]


    init_balance = 10000
    symbols = ['SPY','QQQ','IWM','VGK','EWJ','EEM','VNQ','GLD','DBC','HYG','LQD','TLT','IEF']
    df = get_stock_data_with_ma(symbols=symbols, start_date='2010-01-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [12])

    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    safe_asset_ratio = [0, 0.16, 0.3333, 0.5, 0.6667, 0.8333, 1]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_selected_assets, axis=1) 
    df['Restart Balance'] = None
    df['Safe Asset Ratio'] = None

    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']
        safe_count = len(symbols) -1 -len(restart_asset)

        if len(safe_asset_ratio) <= safe_count: # 모두 안전자산 비율로
            safe_ratio = 1
        else:
            safe_ratio = safe_asset_ratio[safe_count]

        unsafe_ratio = 1 - safe_ratio

        if safe_ratio == 1:
            df.at[idx, 'Restart Asset'] = ['IEF']
        else:
            df.at[idx, 'Restart Asset'] = df.at[idx, 'Restart Asset'][:6] + ['IEF']

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance

        restart_balance = []
        if unsafe_ratio > 0:
            unsafe_balance = balance*unsafe_ratio
            for _ in range(6):
                restart_balance.append(int(unsafe_balance/6))

        if safe_ratio > 0:
            restart_balance.append(int(balance*safe_ratio))
        else:
            restart_balance.append(0)

        df.at[idx, 'Restart Balance'] = restart_balance
        df.at[idx, 'Safe Asset Ratio'] = safe_ratio
            
    # display(df)
    df = get_performance(df)    
    display(df)

paa()

,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...",2011-01-31,2025-03-28,10000,13937,0.023709,0.065042,0.085469,-0.244312


In [18]:
def get_momentum_score(symbol_dfs:dict, intervals=[1,3,6]):

    result_dfs = {}
    max_interval = max(intervals)
    

    for symbol, df in symbol_dfs.items():
        
        interval_dfs ={}
        for interval in intervals:
            copy_df = df.copy()
            shift_df = copy_df.shift(interval)
            
            copy_df['Interval'] = interval

            profit_value = copy_df['Result Close'] - shift_df['Result Close']

            copy_df['Profit Value'] = profit_value
            copy_df['Profit Ratio'] = profit_value/shift_df['Result Close']

            interval_dfs[interval] = copy_df
            
        
        weighted_scores = [
            (df_interval['Profit Ratio'] * (max_interval / interval)).round(2)
            for interval, df_interval in interval_dfs.items()
        ]

        # 리스트 형태로 각 row의 interval 점수를 묶기
        momentum_score_list = [list(t) for t in zip(*[s.values for s in weighted_scores])]

        new_df = df.copy()
        new_df['Momentum Score'] = momentum_score_list
        new_df['Total Momentum Score'] = [sum(x) for x in momentum_score_list]
        new_df['Interval'] = str(intervals)

        result_dfs[symbol] = new_df


    return result_dfs






In [19]:
# VAA 공격형
# 공격형 자산 : SPY, EFA(선진국주식), EEM(개발도상국 주식), AGG(미국혼합 채권)
# 안전 자산 : LQD(미국회사채), IEF(미국중기채), SHY(미국단기채)
# 매월말 공격형, 안전자산 모멘텀 스코어 계산
# 각 자산의 모멘텀 스코어 계산
# 공격형 자산 4개 모두의 모멘텀 스코어가 0이상일 경우 포트폴리오 전체를 가장 모멘텀 스코어가 높은 공격형 자산에 투자
# 공격형 자산 4개 모두의 모멘텀 스코어가 0이하일 경우 포트폴리오 전테를 가장 모멘텀 스코어가 낮은 안전자산에 투자

def vaa_aggressive():

    aggressive_assets = ['SPY', 'EFA', 'EEM', 'AGG']
    safe_assets = ['LQD', 'IEF', 'SHY']
    
    def get_selected_assets(row, top_n=1):
        symbols = row['Symbol']
        scores = row['Total Momentum Score']

        score_dict = dict(zip(symbols, scores))

        aggressive_assets = ['SPY', 'EFA', 'EEM', 'AGG']
        safe_assets = ['LQD', 'IEF', 'SHY']

        aggressive_scores = {sym: score_dict[sym] for sym in aggressive_assets}
        safe_scores = {sym: score_dict[sym] for sym in safe_assets}

        if all(score >= 0 for score in aggressive_scores.values()):
            # 공격형 자산 중 상위 top_n 선택
            selected = sorted(
                aggressive_scores.items(), key=lambda x: x[1], reverse=True
            )[:top_n]
        else:
            # 안전 자산 중 상위 top_n 선택
            selected = sorted(
                safe_scores.items(), key=lambda x: x[1], reverse=True
            )[:top_n]

        return [sym for sym, score in selected]
        
        
    init_balance = 10000
    symbols = aggressive_assets + safe_assets
    df = get_stock_data_with_ma(symbols=symbols, start_date='2010-01-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_momentum_score(df, [1,3,6,12])

    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Total Momentum Score'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval', 'Momentum Score']]
    df = df[columns]


    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_selected_assets, axis=1) 
    df['Restart Balance'] = None
    df['Restart Type'] = None
    
    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance

        split_balance = int(balance / len(restart_asset))
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance
            
    display(df)
    df = get_performance(df)
    display(df)

vaa_aggressive()

,Date,Symbol,Result Close,MMA_10,Total Momentum Score,End Balance,Balance,Restart Asset,Restart Balance,Restart Type
0,2011-01-31,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[128.68, 59.44, 45.81, 105.66, 108.48, 93.8, 8...","[115.25, 53.49, 42.8, 107.06, 109.7, 95.81, 84...","[1.15, 0.84, -0.08, -0.15, -0.14, -0.22, 0.01]",None,10000,[SHY],[10000],None
1,2011-02-28,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[133.15, 61.55, 45.79, 105.65, 109.22, 93.37, ...","[116.69, 54.21, 43.18, 107.14, 109.89, 96.07, ...","[1.64, 1.6, 0.55, -0.11, 0.01, -0.33, -0.05]",[9979],9979,[LQD],[9979],None
2,2011-03-31,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[132.59, 60.08, 48.67, 105.13, 108.2, 93.01, 8...","[119.01, 55.38, 44.24, 107.08, 110.17, 96.07, ...","[0.62, 0.1, 1.17, -0.13, -0.19, -0.16, -0.04]",[9885],9885,[SHY],[9885],None
3,2011-04-29,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[136.43, 63.46, 50.0, 106.46, 110.47, 94.47, 8...","[122.33, 57.08, 45.5, 107.0, 110.37, 95.95, 84...","[1.04, 1.35, 1.06, 0.15, 0.32, 0.17, 0.05]",[9929],9929,[EFA],[9929],None
4,2011-05-31,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[134.9, 62.06, 48.53, 107.46, 111.44, 96.59, 8...","[124.79, 58.09, 46.22, 106.96, 110.48, 95.98, ...","[0.43, 0.34, 0.33, 0.2, 0.27, 0.43, 0.06]",[9709],9709,[SPY],[9709],None
5,2011-06-30,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[131.97, 60.14, 47.6, 106.67, 110.13, 95.86, 8...","[127.46, 59.11, 46.97, 106.74, 110.2, 95.64, 8...","[0.1, -0.01, -0.04, -0.02, -0.02, 0.07, 0.03]",[9498],9498,[IEF],[9498],None
6,2011-07-29,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[129.33, 58.71, 47.11, 108.16, 112.37, 98.66, ...","[128.98, 59.49, 47.2, 106.69, 110.12, 95.6, 84...","[-0.27, -0.48, -0.15, 0.28, 0.4, 0.65, 0.06]",[9775],9775,[IEF],[9775],None
7,2011-08-31,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[122.22, 53.57, 42.75, 109.5, 112.33, 103.0, 8...","[129.35, 59.15, 46.87, 106.78, 110.12, 96.02, ...","[-1.04, -1.79, -1.65, 0.31, 0.08, 1.05, 0.07]",[10204],10204,[IEF],[10204],None
8,2011-09-30,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[113.15, 47.78, 35.1, 110.11, 112.31, 105.07, ...","[128.82, 58.5, 45.9, 107.06, 110.34, 96.77, 84...","[-1.76, -2.66, -3.98, 0.3, 0.15, 0.94, 0.01]",[10409],10409,[IEF],[10409],None
9,2011-10-31,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]","[125.5, 52.38, 40.81, 110.03, 114.7, 103.51, 8...","[128.79, 57.92, 45.22, 107.48, 110.97, 97.73, ...","[1.09, 0.3, 0.93, 0.14, 0.44, 0.26, 0.01]",[10254],10254,[SPY],[10254],None


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, EFA, EEM, AGG, LQD, IEF, SHY]",2011-01-31,2025-03-28,10000,18781,0.045493,0.072582,0.374344,-0.183895


In [20]:
# VAA 중도형
# 공격형자산 : SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, HYG, LQD, TLT
# 안전자산 : LQD, IEF, SHY
# (LQD 중복인데, 이거 확인해봐야할 듯)
# 공격형자산 중 모멘텀 스코어가 0 이하인 자산의 개수를 측정
# 하락형 자산이 4개 이상일 경우에는 100% 안전자산 투자
# 안전자산에 투자하지 않은 자금은 상대모멘텀을 적용해 5개의 ETF에 투자 - 상대모멘텀이 가장 높은 5개


def vaa_balance():
    aggressive_assets = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'GLD', 'DBC', 'HYG', 'LQD', 'TLT']
    safe_assets = ['LQD', 'IEF', 'SHY']
    
    def get_selected_assets(row, top_n=None):  # top_n=None이면 전체 추출
        symbols = row['Symbol']
        scores = row['Total Momentum Score']

        score_dict = dict(zip(symbols, scores))

        aggressive_scores = {sym: score_dict[sym] for sym in aggressive_assets if score_dict[sym] >= 0}
        safe_scores = {sym: score_dict[sym] for sym in safe_assets}

        selected_scores = []
        if aggressive_scores:
            selected_scores = sorted(aggressive_scores.items(), key=lambda x: x[1], reverse=True)
        # else:
        #    selected_scores = sorted(safe_scores.items(), key=lambda x: x[1], reverse=True)

        # top_n이 None이면 전체, 아니면 top_n개만
        if top_n is not None and top_n > 0:
            selected_scores = selected_scores[:top_n]

        return [sym for sym, score in selected_scores]
        
        
    init_balance = 10000
    symbols = aggressive_assets + safe_assets
    df = get_stock_data_with_ma(symbols=symbols, start_date='2010-01-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_momentum_score(df, [1,3,6,12])

    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Total Momentum Score'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval', 'Momentum Score']]
    df = df[columns]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_selected_assets, axis=1)
    df['Restart Balance'] = None
    df['Safe Ratio'] = None

    safe_asset_ratio = [0, 0.25, 0.5, 0.75, 1]

    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']
        safe_count = len(aggressive_assets) - len(restart_asset)
        
        if len(safe_asset_ratio) <= safe_count:
            safe_ratio = 1
        else:
            safe_ratio = safe_asset_ratio[safe_count]

        symbols = df.at[idx, 'Symbol']
        scores  = df.at[idx, 'Total Momentum Score']
        score_dict = dict(zip(symbols, scores))
        safe_scores = {sym: score_dict[sym] for sym in safe_assets}
        selected_scores = sorted(safe_scores.items(), key=lambda x: x[1], reverse=True)
        restart_safe_asset = [sym for sym, score in selected_scores[:1]]

        df.at[idx, 'Safe Ratio'] = safe_ratio

        if safe_ratio == 1:
            df.at[idx, 'Restart Asset'] = restart_safe_asset
        else:
            restart_asset = [unsafe_asset for unsafe_asset in restart_asset[:5]]
            
            if safe_ratio > 0:
                df.at[idx,'Restart Asset'] = restart_asset + restart_safe_asset
            else:
                df.at[idx,'Restart Asset'] = restart_asset

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance
        
        unsafe_balance = balance * (1-safe_ratio)
        safe_balance = balance * safe_ratio

        restart_balance = []
        if unsafe_balance > 0:
            for _ in range(len(restart_asset)):
                restart_balance.append(int(unsafe_balance/len(restart_asset)))

        if safe_balance > 0:
            restart_balance.append(int(safe_balance))

        df.at[idx, 'Restart Balance'] = restart_balance
        
    display(df)

    df = get_performance(df)
    display(df)


vaa_balance()
    

,Date,Symbol,Result Close,MMA_10,Total Momentum Score,End Balance,Balance,Restart Asset,Restart Balance,Safe Ratio
0,2011-01-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[128.68, 56.0, 77.95, 50.9, 43.72, 45.81, 57.1...","[115.25, 49.06, 69.1, 46.4, 40.05, 42.8, 52.23...","[1.15, 1.38, 1.1, 0.76, 0.76, -0.08, 1.17, -0....",None,10000,[SHY],[10000],1
1,2011-02-28,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[133.15, 57.77, 82.27, 52.49, 46.12, 45.79, 59...","[116.69, 49.91, 70.16, 47.01, 40.51, 43.18, 52...","[1.64, 1.77, 2.23, 1.39, 1.77, 0.55, 1.75, 1.3...",[9979],9979,"[IWM, DBC, QQQ, EWJ, VNQ, LQD]","[1496, 1496, 1496, 1496, 1496, 2494]",0.25
2,2011-03-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[132.59, 57.43, 84.17, 51.95, 41.28, 48.67, 58...","[119.01, 51.1, 71.95, 48.12, 40.83, 44.24, 53....","[0.62, 0.68, 1.31, 0.32, -1.4, 1.17, 0.39, 0.6...","[1530, 1535, 1487, 1339, 1461, 2470]",9822,"[DBC, IWM, EEM, GLD, QQQ, SHY]","[491, 491, 491, 491, 491, 7366]",0.75
3,2011-04-29,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[136.43, 59.08, 86.39, 56.27, 42.12, 50.0, 61....","[122.33, 52.74, 74.48, 49.73, 41.36, 45.5, 55....","[1.04, 1.02, 1.42, 1.84, 0.2, 1.06, 1.47, 2.38...","[513, 503, 504, 534, 505, 7399]",9958,"[GLD, DBC, VGK, VNQ, IWM]","[1991, 1991, 1991, 1991, 1991]",0
4,2011-05-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[134.9, 58.36, 84.84, 54.67, 41.08, 48.53, 62....","[124.79, 53.99, 76.46, 50.61, 41.62, 46.22, 56...","[0.43, 0.41, 0.51, 0.49, -0.66, 0.33, 0.96, 0....","[1955, 1888, 1934, 2018, 1955]",9750,"[VNQ, GLD, TLT, IWM, VGK, IEF]","[1462, 1462, 1462, 1462, 1462, 2437]",0.25
5,2011-06-30,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[131.97, 57.05, 82.8, 53.53, 41.72, 47.6, 60.1...","[127.46, 55.35, 78.72, 51.58, 42.04, 46.97, 57...","[0.1, 0.14, 0.11, 0.38, 0.27, -0.04, 0.08, 0.1...","[1401, 1426, 1422, 1426, 1431, 2418]",9524,[IEF],[9524],1
6,2011-07-29,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[129.33, 58.0, 79.74, 51.07, 42.84, 47.11, 61....","[128.98, 56.24, 79.94, 51.82, 42.37, 47.2, 58....","[-0.27, 0.47, -0.47, -0.8, 0.46, -0.15, 0.48, ...",[9802],9802,[IEF],[9802],1
7,2011-08-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[122.22, 55.06, 72.65, 46.33, 39.4, 42.75, 57....","[129.35, 56.53, 80.18, 51.34, 42.3, 46.87, 58....","[-1.04, -0.66, -1.66, -1.89, -1.36, -1.65, -0....",[10233],10233,[IEF],[10233],1
8,2011-09-30,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[113.15, 52.49, 64.3, 40.66, 37.84, 35.1, 50.8...","[128.82, 56.57, 79.33, 50.7, 41.98, 45.9, 58.5...","[-1.76, -0.98, -2.79, -3.03, -1.06, -3.98, -2....",[10438],10438,[IEF],[10438],1
9,2011-10-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[125.5, 57.95, 74.01, 45.71, 37.6, 40.81, 58.1...","[128.79, 56.92, 78.91, 50.36, 41.37, 45.22, 58...","[1.09, 1.32, 1.28, 0.58, -0.84, 0.93, 1.47, 1....",[10283],10283,"[VNQ, GLD, QQQ, IWM, SPY, LQD]","[1542, 1542, 1542, 1542, 1542, 2570]",0.25


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...",2011-01-31,2025-03-28,10000,12990,0.018637,0.057066,0.001689,-0.210436


In [21]:
def calc_balance(df, idx, init_balance):
    if idx == 0:
        balance = init_balance
        end_balance = []  # 첫 번째 행에는 개별 자산 정보가 없을 수도 있으므로 빈 리스트
    else:
        prev_restart_balance = df.at[idx-1, 'Restart Balance']
        prev_restart_asset = df.at[idx-1, 'Restart Asset']

        end_balance = []
        for i in range(len(prev_restart_balance)):
            symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
            prev_close = df.at[idx-1, 'Result Close'][symbol_idx]
            curr_close = df.at[idx, 'Result Close'][symbol_idx]

            change_rate = (curr_close - prev_close) / prev_close
            symbol_val = int(prev_restart_balance[i] * (1 + change_rate))
            end_balance.append(symbol_val)

        balance = sum(end_balance)

    return balance, end_balance
    

In [25]:
# DAA 전략
# 카나리아 자산군 (WWO, BND으로 공격형 자산 비중을 맞춤
# 안전자산 비중을 가장 모멘텀 스코어가 높은 안전자산에 투자
# 안전자산에 투자하지 않은 자금은 상대 모멘텀을 적용해 6개 ETF에 투자
# 매월 말 각 ETF의 모멘텀 스코어 계산, 가장 높은 6개 ETF에 동일한 금액 투자
def daa():
    aggressive_assets = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'GLD', 'DBC', 'HYG', 'LQD', 'TLT']
    safe_assets = ['LQD', 'IEF', 'SHY']
    kanaria_assets = ['VWO','BND']

    init_balance = 10000
    symbols = aggressive_assets + safe_assets + kanaria_assets
    df = get_stock_data_with_ma(symbols=symbols, start_date='2010-01-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_momentum_score(df, [1,3,6,12])

    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Total Momentum Score'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval', 'Momentum Score']]
    df = df[columns]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = None
    df['Restart Balance'] = None
    df['Safe Ratio'] = None

    for idx in range(len(df)):
        symbols = df.at[idx, 'Symbol']
        scores  = df.at[idx, 'Total Momentum Score']
        score_dict = dict(zip(symbols, scores))

        safe_score = 0
        if score_dict['VWO'] > 0:
            safe_score += 1
        if score_dict['BND'] > 0:
            safe_score += 1

        if safe_score == 2:
            safe_ratio = 1
        elif safe_score == 1:
            safe_ratio = 0.5
        else:
            safe_ratio = 0

        # 안전비율
        df.at[idx, 'Safe Ratio'] = safe_ratio

        # 안전자산 구하기
        if score_dict['VWO'] >= score_dict['BND']:
            higher_safe_symbol = 'VWO'
        else:
            higher_safe_symbol = 'BND'
        

        safe_assets = []
        unsafe_assets = []

        if safe_ratio > 0:
            safe_assets = [higher_safe_symbol]

        if safe_ratio < 1:
            aggressive_scores = {sym: score_dict[sym] for sym in aggressive_assets}
            selected_scores = []
            if aggressive_scores:
                selected_scores = sorted(aggressive_scores.items(), key=lambda x: x[1], reverse=True)
            
            unsafe_assets = [sym for sym, score in selected_scores[:5]]


        # 리벨런싱 자산
        df.at[idx, 'Restart Asset'] = unsafe_assets + safe_assets

        balance, end_balance = calc_balance(df, idx, init_balance)
        
        df.at[idx, 'End Balance'] = end_balance
        df.at[idx, 'Balance'] = balance

        unsafe_balance = int(balance *(1-safe_ratio))
        safe_balance = balance - unsafe_balance

        restart_balance = []
        if len(unsafe_assets) > 0:
            for _ in range(len(unsafe_assets)):
                restart_balance.append(int(unsafe_balance/len(unsafe_assets)))

        if len(safe_assets) > 0:
            for _ in range(len(safe_assets)):
                restart_balance.append(int(safe_balance/len(safe_assets)))

        df.at[idx, 'Restart Balance'] = restart_balance
        
    display(df)
    df = get_performance(df)
    display(df)
        
        
daa()

,Date,Symbol,Result Close,MMA_10,Total Momentum Score,End Balance,Balance,Restart Asset,Restart Balance,Safe Ratio
0,2011-01-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[128.68, 56.0, 77.95, 50.9, 43.72, 45.81, 57.1...","[115.25, 49.06, 69.1, 46.4, 40.05, 42.8, 52.23...","[1.15, 1.38, 1.1, 0.76, 0.76, -0.08, 1.17, -0....",[],10000,"[DBC, QQQ, VNQ, SPY, IWM]","[2000, 2000, 2000, 2000, 2000]",0
1,2011-02-28,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[133.15, 57.77, 82.27, 52.49, 46.12, 45.79, 59...","[116.69, 49.91, 70.16, 47.01, 40.51, 43.18, 52...","[1.64, 1.77, 2.23, 1.39, 1.77, 0.55, 1.75, 1.3...","[2083, 2063, 2094, 2069, 2110]",10419,"[IWM, DBC, QQQ, EWJ, VNQ, VWO]","[1041, 1041, 1041, 1041, 1041, 5210]",0.5
2,2011-03-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[132.59, 57.43, 84.17, 51.95, 41.28, 48.67, 58...","[119.01, 51.1, 71.95, 48.12, 40.83, 44.24, 53....","[0.62, 0.68, 1.31, 0.32, -1.4, 1.17, 0.39, 0.6...","[1065, 1068, 1034, 931, 1016, 5495]",10609,"[DBC, IWM, EEM, GLD, QQQ, VWO]","[1060, 1060, 1060, 1060, 1060, 5305]",0.5
3,2011-04-29,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[136.43, 59.08, 86.39, 56.27, 42.12, 50.0, 61....","[122.33, 52.74, 74.48, 49.73, 41.36, 45.5, 55....","[1.04, 1.02, 1.42, 1.84, 0.2, 1.06, 1.47, 2.38...","[1108, 1087, 1088, 1154, 1090, 5483]",11010,[VWO],[11010],1
4,2011-05-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[134.9, 58.36, 84.84, 54.67, 41.08, 48.53, 62....","[124.79, 53.99, 76.46, 50.61, 41.62, 46.22, 56...","[0.43, 0.41, 0.51, 0.49, -0.66, 0.33, 0.96, 0....",[10685],10685,[VWO],[10685],1
5,2011-06-30,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[131.97, 57.05, 82.8, 53.53, 41.72, 47.6, 60.1...","[127.46, 55.35, 78.72, 51.58, 42.04, 46.97, 57...","[0.1, 0.14, 0.11, 0.38, 0.27, -0.04, 0.08, 0.1...",[10578],10578,"[VGK, EWJ, GLD, QQQ, IWM, VWO]","[1057, 1057, 1057, 1057, 1057, 5289]",0.5
6,2011-07-29,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[129.33, 58.0, 79.74, 51.07, 42.84, 47.11, 61....","[128.98, 56.24, 79.94, 51.82, 42.37, 47.2, 58....","[-0.27, 0.47, -0.47, -0.8, 0.46, -0.15, 0.48, ...","[1008, 1085, 1145, 1074, 1017, 5256]",10585,"[GLD, DBC, TLT, VNQ, QQQ, BND]","[1058, 1058, 1058, 1058, 1058, 5293]",0.5
7,2011-08-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[122.22, 55.06, 72.65, 46.33, 39.4, 42.75, 57....","[129.35, 56.53, 80.18, 51.34, 42.3, 46.87, 58....","[-1.04, -0.66, -1.66, -1.89, -1.36, -1.65, -0....","[1187, 1053, 1156, 998, 1004, 5366]",10764,"[GLD, TLT, DBC, LQD, QQQ, BND]","[1076, 1076, 1076, 1076, 1076, 5382]",0.5
8,2011-09-30,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[113.15, 52.49, 64.3, 40.66, 37.84, 35.1, 50.8...","[128.82, 56.57, 79.33, 50.7, 41.98, 45.9, 58.5...","[-1.76, -0.98, -2.79, -3.03, -1.06, -3.98, -2....","[956, 1214, 918, 1075, 1025, 5403]",10591,"[TLT, LQD, GLD, QQQ, EWJ, BND]","[1059, 1059, 1059, 1059, 1059, 5296]",0.5
9,2011-10-31,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...","[125.5, 57.95, 74.01, 45.71, 37.6, 40.81, 58.1...","[128.79, 56.92, 78.91, 50.36, 41.37, 45.22, 58...","[1.09, 1.32, 1.28, 0.58, -0.84, 0.93, 1.47, 1....","[1015, 1081, 1121, 1169, 1052, 5288]",10726,[VWO],[10726],1


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, QQQ, IWM, VGK, EWJ, EEM, VNQ, GLD, DBC, ...",2011-01-31,2025-03-28,10000,17716,0.041194,0.109578,0.241181,-0.27024
